In [20]:
import pandas as pd
from IPython.display import HTML
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

/opt/anaconda3/envs/ESTsoft_TP_1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df_type = pd.read_csv('../../data/preprocessed_data/시도_시군구별_보행자_사고_사고유형_전처리ver.csv')
# df_type
HTML(df_type.head(5).to_html())

# 원핫 인코딩 > y값을 위험도로
# 위험한 동을 예측

# 포아송 회귀 시도

,연도,시군구,법정동코드,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,도로형태,피해자 상해정도
0,2020,서울특별시 종로구 창성동,1111010500,중상사고,0,1,0,0,차대사람 - 차도통행중,교차로 - 교차로부근,중상
1,2020,서울특별시 종로구 창성동,1111010500,중상사고,0,1,0,0,차대사람 - 차도통행중,교차로 - 교차로안,중상
2,2020,서울특별시 종로구 통인동,1111010800,경상사고,0,0,1,0,차대사람 - 기타,단일로 - 기타,경상
3,2020,서울특별시 종로구 누상동,1111010900,경상사고,0,0,1,0,차대사람 - 차도통행중,교차로 - 교차로안,경상
4,2020,서울특별시 종로구 누상동,1111010900,경상사고,0,0,1,0,차대사람 - 기타,단일로 - 기타,경상


In [13]:
df_type['사고유형'].unique()

array(['차대사람 - 차도통행중', '차대사람 - 기타', '차대사람 - 횡단중', '차대사람 - 보도통행중',
       '차대사람 - 길가장자리구역통행중'], dtype=object)

In [14]:
df_type['도로형태'].unique()

array(['교차로 - 교차로부근', '교차로 - 교차로안', '단일로 - 기타', '기타 - 기타',
       '교차로 - 교차로횡단보도내', '단일로 - 지하차도(도로)내', '주차장 - 주차장', '단일로 - 고가도로위',
       '단일로 - 교량', '미분류 - 미분류', '단일로 - 터널'], dtype=object)

In [ ]:
# 앞에서 불러온 CSV 파일을 기준으로 재지정
df = df_type.copy()

# 피해 가중 점수 계산
df_risk = df.copy()
df_risk['피해가중점수'] = (
    df_risk['사망자수'] * 10 +
    df_risk['중상자수'] * 5 +
    df_risk['경상자수'] +
    df_risk['부상신고자수']
)
# 보험사 시준 탐색


# 도로형태_가중치 = 사고유형별_누적_피해점수 / 전체_평균_피해점수
type_weight = df_risk.groupby('사고유형')['피해가중점수'].mean()
df_risk['사고유형가중치'] = df_risk['사고유형'].map(type_weight)

# 도로형태_가중치 = 도로형태별_누적_피해점수 / 전체_평균_피해점수
road_weight = df_risk.groupby('도로형태')['피해가중점수'].mean()
df_risk['도로형태가중치'] = df_risk['도로형태'].map(road_weight)

# 최종 위험 점수 계산 (기본 피해 점수 + 유형/도로 가중치 평균)
df_risk['최종위험점수'] = df_risk['피해가중점수'] + 0.5 * (df_risk['사고유형가중치'] + df_risk['도로형태가중치'])
# df_risk['최종위험점수'] = df_risk['사고유형가중치'] + df_risk['도로형태가중치']

# 위험도 5등급 (1=매우 안전 ~ 5=매우 위험)
df_risk['위험도'] = pd.qcut(df_risk['최종위험점수'], q=5, labels=[1, 2, 3, 4, 5])

# 결과 테이블 요약
columns_to_show = ['연도', '시군구', '법정동코드', '사고유형', '도로형태', '피해가중점수', '사고유형가중치', '도로형태가중치', '최종위험점수', '위험도']
# df_risk[columns_to_show]
HTML(df_risk[columns_to_show].head(5).to_html(index=False))

# 저장
# df_risk.to_csv('data/03_modeling/modeling_2_risk.csv', index=False)

연도,시군구,법정동코드,사고유형,도로형태,피해가중점수,사고유형가중치,도로형태가중치,최종위험점수,위험도
2020,서울특별시 종로구 창성동,1111010500,차대사람 - 차도통행중,교차로 - 교차로부근,5,3.861877,3.767438,8.814657,4
2020,서울특별시 종로구 창성동,1111010500,차대사람 - 차도통행중,교차로 - 교차로안,5,3.861877,3.856756,8.859317,4
2020,서울특별시 종로구 통인동,1111010800,차대사람 - 기타,단일로 - 기타,1,3.427611,3.767168,4.597389,1
2020,서울특별시 종로구 누상동,1111010900,차대사람 - 차도통행중,교차로 - 교차로안,1,3.861877,3.856756,4.859317,2
2020,서울특별시 종로구 누상동,1111010900,차대사람 - 기타,단일로 - 기타,1,3.427611,3.767168,4.597389,1


In [28]:
# 사용할 feature와 target 정의
features = ['시군구', '사고유형', '도로형태']
target = '위험도'

# 위험도 타입 설정
df_risk[target] = df_risk[target].astype(str)

# 학습/테스트 분할
train_data, test_data = train_test_split(df_risk[features + [target]], test_size=0.2, random_state=42, stratify=df_risk[target])

# AutoGluon 학습
predictor = TabularPredictor(label=target, problem_type='multiclass', eval_metric='accuracy')
predictor.fit(train_data)

# 성능 평가
predictor.evaluate(test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20250604_074927"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.4.0: Fri Apr 11 18:33:39 PDT 2025; root:xnu-11417.101.15~117/RELEASE_ARM64_T6020
CPU Count:          12
Memory Avail:       3.19 GB / 16.00 GB (19.9%)
Disk Space Avail:   29.47 GB / 460.43 GB (6.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize acc

Beginning AutoGluon training ...
AutoGluon will save models to "/Users/leehyunseung/Documents/GitHub/1_team_project/notebooks/03_modeling/AutogluonModels/ag-20250604_074927"
Train Data Rows:    41336
Train Data Columns: 3
Label Column:       위험도
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 5
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3213.73 MB
	Train Data (Original)  Memory Usage: 15.65 MB (0.5% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropD

{'accuracy': 0.5553942912433478,
 'balanced_accuracy': 0.555932141962107,
 'mcc': 0.4568031729297429}

In [29]:
# 저장
predictor.save("risk_predictor_model")

# 예측
sample = pd.DataFrame([{
    '시군구': '종로구',
    '사고유형': '차대사람 - 차도통행중',
    '도로형태': '교차로 - 교차로부근'
}])
predictor.predict(sample)

0    4
Name: 위험도, dtype: object